In [104]:
import pandas as pd
import os
import requests

In [105]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
newRDir   = '/Users/amcguire/Library/Application Support/KoLmafia/data/test/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'
multiUser = {'madowl (combined)':['themadowl','madowl','Arkcon']}

clanMap   = {'The Hogs of Exploitery':'HoE', 
             'KirbyLlama\'s Private Dungeon Clan': 'KirbyClan',
             'The Piglets of Fate': 'Piglets',
             'The Hogs of Destiny': 'HoD'}

runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16-3496','r17-4041','r18-3582',
             'r19-4318','r20-3584','r21-3794','r22-4698','r25-3802']
             #'r24p-1852','r23p-722','r26p-900','r27p-2770']

In [106]:
# Temporary script to convert Stary script output into old c/p format

for file in os.listdir(newRDir):
    
    clan = file.split(sep='_')[1]
    rNum = file.split(sep='_')[2][:6]
    newFileName = 'r{1}_{0}.txt'.format(clanMap[clan],rNum)
    
    currFile = open('{}/{}'.format(newRDir, file)).read()
    
    newFile = ''
    
    for ct, line in enumerate(currFile.split(sep='\n')):
        try:     newFile = newFile + line.split(sep='\t')[1] + '\n'
        except:  newFile = newFile + 'LINE-ERROR' + '\n'
    
    print('{} file parsed. {} new lines.'.format(clan,ct))
    
    file = open('{}{}'.format(raidDir,newFileName),"w")
    
    file.write(newFile)
    
    file.close()
    
    runFiles = runFiles + [newFileName[:-4]]

The Hogs of Exploitery file parsed. 67 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 145 new lines.
The Piglets of Fate file parsed. 231 new lines.
The Hogs of Destiny file parsed. 139 new lines.


In [107]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [108]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
            
collectRaiders(allRuns)

['Name Guy Man',
 'themadowl',
 'violetinsane',
 'HeroBill',
 'Stary',
 'playultm8',
 'Liora',
 'Volotani',
 'Captain Scotch',
 'kenny kamAKAzi',
 'madowl',
 'Phillammon',
 'kirByllAmA',
 'Lyft',
 'NukebOy1313',
 'RandomExtremity',
 'gregmasta',
 'AeNimUS',
 'chameco',
 'Gausie',
 'Just Eyes',
 'John Keel',
 'threebullethamburgler',
 '<b>NukebOy1313',
 'tdog21',
 'Arkcon',
 '<b>Captain Scotch']

In [109]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [110]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})
            
tabulateLoots('Captain Scotch', allRuns)

{'loot': ["Drunkula's wineglass",
  'zombie accordion',
  "Mayor Ghost's cloak",
  "Drunkula's bell",
  "Mayor Ghost's toupee",
  "Great Wolf's headband",
  "Mayor Ghost's sash",
  "Great Wolf's headband",
  "Unkillable Skeleton's restless leg",
  "Unkillable Skeleton's restless leg",
  'Quiets-Your-Steps',
  'Gets-You-Drunk',
  'HOA zombie eyes',
  'Gets-You-Drunk',
  "Drunkula's ring of haze",
  "Mayor Ghost's scissors",
  'Helps-You-Sleep'],
 'skills': 11}

In [111]:
finalDisplay = {}

# Create summary dictionary via tabulation code

for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])

In [112]:
# Slice of code for combining duplicate users

for name in multiUser.keys():
    
    finalDisplay[name] = {}
    
    for field in ['kills','defeats','loot','skills','lootCount']:
        try:    finalDisplay[name][field] = sum([finalDisplay[x][field] for x in multiUser[name]])
        except: finalDisplay[name][field] = [].append([finalDisplay[x][field] for x in multiUser[name]])
            
    for x in multiUser[name]: finalDisplay.pop(x)

In [113]:
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame.sort_values(by='kill/skillz', ascending=False)

,kills,defeats,skills,lootCount,loot,kill/skillz
playultm8,1130,0,0,0,[],2260.000000
kenny kamAKAzi,4888,1,3,4,"[Mayor Ghost's sash, Mayor Ghost's gavel, Quie...",1396.571429
Just Eyes,7493,92,5,7,"[Gets-You-Drunk, Drunkula's wineglass, Protect...",1362.363636
Volotani,4650,61,3,12,"[Mayor Ghost's khakis, Mayor Ghost's scissors,...",1328.571429
madowl (combined),10228,269,8,20,None,1203.294118
gregmasta,2982,25,2,11,"[Gets-You-Drunk, Unkillable Skeleton's shingua...",1192.800000
NukebOy1313,2845,7,2,2,"[Gets-You-Drunk, Drunkula's wineglass]",1138.000000
RandomExtremity,2838,5,2,7,"[Drunkula's bell, Great Wolf's rocket launcher...",1135.200000
Captain Scotch,12765,35,11,17,"[Drunkula's wineglass, zombie accordion, Mayor...",1110.000000
Liora,548,7,0,1,[Great Wolf's left paw],1096.000000


In [114]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 67 skills for 25 Loathers. Good job folks!


In [115]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [100]:
n = "themadowl"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 331, 'defeats': 56}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 765, 'defeats': 2}
r6-3598 - {'kills': 661, 'defeats': 0}
r7-4372 - {'kills': 71, 'defeats': 0}
r8-3473 - {'kills': 289, 'defeats': 0}
r9-3858 - {'kills': 1031, 'defeats': 1}
r10-3787 - {'kills': 385, 'defeats': 0}
r11-3797 - {'kills': 0, 'defeats': 0}
r12-3798 - {'kills': 0, 'defeats': 0}
r13-12260 - {'kills': 1, 'defeats': 0}
r14-8393 - {'kills': 377, 'defeats': 1}
r15-3769 - {'kills': 380, 'defeats': 1}
r16-3496 - {'kills': 0, 'defeats': 0}
r17-4041 - {'kills': 851, 'defeats': 5}
r18-3582 - {'kills': 0, 'defeats': 0}
r19-4318 - {'kills': 0, 'defeats': 0}
r20-3584 - {'kills': 0, 'defeats': 0}
r21-3794 - {'kills': 0, 'defeats': 0}
r22-4698 - {'kills': 0, 'defeats': 0}
r25-3802 - {'kills': 0, 'defeats': 0}
r188769_HoE - {'kills': 0, 'defeats': 0}
r188757_KirbyClan - {'kills': 0, 'defeats': 0}
r188579_Piglets - {'kills

In [13]:
n = "gregmasta"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 0, 'defeats': 0}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 0, 'defeats': 0}
r8-3473 - {'kills': 0, 'defeats': 0}
r9-3858 - {'kills': 1, 'defeats': 0}
r10-3787 - {'kills': 0, 'defeats': 0}
r11-3797 - {'kills': 115, 'defeats': 0}
r12-3798 - {'kills': 361, 'defeats': 0}
r13-12260 - {'kills': 752, 'defeats': 5}
r14-8393 - {'kills': 744, 'defeats': 0}
r15-3769 - {'kills': 688, 'defeats': 7}
r16-3496 - {'kills': 0, 'defeats': 0}
r17-4041 - {'kills': 321, 'defeats': 13}
r18-3582 - {'kills': 0, 'defeats': 0}
r19-4318 - {'kills': 0, 'defeats': 0}
r20-3584 - {'kills': 0, 'defeats': 0}
r21-3794 - {'kills': 0, 'defeats': 0}
r22-4698 - {'kills': 0, 'defeats': 0}
r23p-722 - {'kills': 0, 'defeats': 0}
r24p-1852 - {'kills': 0, 'defeats': 0}
r25-3802 - {'kills': 0, 'defeats': 0}
r26p-900 - {'kills': 0, 'defeats': 0}
r27p-2

In [79]:
import numpy as np

def roll1( x, y ):
    if ( x == 2 ) & ( y == 3 ):
        return 1
    else:
        return 0


0    0
1    0
2    0
dtype: int64